In [23]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri

# Load seqICP library
robjects.r('''
    library(seqICP)
''')

pandas2ri.activate()
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tqdm'

In [11]:
target = "SPY"

data_train = pd.read_clipboard()
date = data_train['date']
data_train = data_train.drop('date', axis=1)

data_train.tail()

,SPY,RPI,HWI,DPCERA3M086SBEA,SRVPRD,UEMPLT5,CES2000000008,AMDMUOx,CES0600000007,PERMITS,...,CPIMEDSL,M2REAL,EXJPUSx,S&P: indust,GS5,EXSZUSx,TB6SMFFM,T10YFFM,WPSID62,PCEPI
88,-0.518629,-0.201457,-1.087011,0.165650,-0.638882,-0.147693,-0.121139,1.124335,1.040953,-1.428328,...,2.183389,-0.150529,-0.759995,0.500797,-0.605197,-0.947413,-0.838470,-1.024532,-0.299778,-0.163766
89,-0.822712,-0.201457,-1.087011,0.165650,-0.638882,-0.147693,-0.121139,1.124335,1.040953,-1.428328,...,2.183389,-0.150529,-0.759995,0.500797,-0.605197,-0.947413,-0.838470,-1.024532,-0.299778,-0.163766
90,0.205749,-0.021230,0.863813,-0.320357,0.527758,-0.207087,0.616038,0.398382,1.298422,-1.825222,...,-0.174830,-1.044750,-0.970367,1.044989,-0.952625,-0.623020,-2.211211,-1.129475,0.338182,1.119045
91,1.047655,-0.021230,0.863813,-0.320357,0.527758,-0.207087,0.616038,0.398382,1.298422,-1.825222,...,-0.174830,-1.044750,-0.970367,1.044989,-0.952625,-0.623020,-2.211211,-1.129475,0.338182,1.119045
92,0.060037,-0.490621,0.215310,-0.329593,0.126653,0.181334,-0.489013,0.793376,0.783483,-2.550590,...,1.253821,-0.918135,0.130970,0.943277,0.046230,-0.370354,-1.793420,-1.004856,0.921421,0.662715


In [21]:
data_train = data_train.loc[0:50, :]

# convert dataframe to R objects
X_train = data_train.drop(target, axis=1).values
y_train = data_train[target]

X_train_r = numpy2ri.numpy2ri(X_train)
y_train_r = numpy2ri.numpy2ri(y_train)

# pass inputs to global variables
robjects.globalenv['Xmatrix'] = X_train_r
robjects.globalenv['Y'] = y_train_r

data_train.shape

robjects.r(f'''
    seqICP_result <- seqICP(X = Xmatrix,
                            Y = Y,
                            max.parents = 5,
                            stopIfEmpty = FALSE,
                            silent = TRUE)
    seqICP_summary <- summary(seqICP_result)
    parent_set <- seqICP_result$parent.set

''')

# retrieve results from seqICP
parent_set = robjects.r['parent_set']


 Invariant Linear Causal Regression at level 0.05

 No variable shows a significant causal effect

 
 
         
 coefficient
 lower bound
 upper bound
  p-value

intercept
       -0.15
      -1.211
       0.838
       NA

X1       
        0.00
       0.000
       0.000
        1

X2       
        0.00
       0.000
       0.000
        1

X3       
        0.00
       0.000
       0.000
        1

X4       
        0.00
       0.000
       0.000
        1

X5       
        0.00
       0.000
       0.000
        1

X6       
        0.00
       0.000
       0.000
        1

X7       
        0.00
       0.000
       0.000
        1

X8       
        0.00
       0.000
       0.000
        1

X9       
        0.00
       0.000
       0.000
        1

X10      
        0.00
       0.000
       0.000
        1

X11      
        0.00
       0.000
       0.000
        1

X12      
        0.00
       0.000
       0.000
        1

X13      
        0.00
       0.000
       0.000
       

In [22]:
parent_set

array([], dtype=float64)